# Import Required Packages

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss, BCELoss

import glob
import csv
import cv2
from numpy import array, asarray, ndarray, swapaxes
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
#!pip install torchvision
#!pip install opencv-python

# Download Files

In [3]:
#training controls
batch_size = 64
epochs = 10
training_size = 0.8
learning_rate = 0.0001

# input image dimensions
img_rows, img_cols = 268, 182

In [4]:
# data holders
x_test = []
x_train = []
y_test= []
y_train= []
tempY = []

In [5]:
# opening the dataset
dataset = csv.reader(open("MovieGenre417.csv",encoding="utf8",errors='replace'), delimiter=",")

# skipping the header line
next(dataset)

['', 'index', 'imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster']

In [6]:
# extract images from zip folder

import zipfile as zf

files = zf.ZipFile("FinalProjectUp.zip", 'r')
files.extractall()
files.close()

In [7]:
# list of image files in SampleMoviePosters folder
flist=glob.glob('FinalProject/*.jpg')  

In [8]:
len(flist)

5384

In [9]:
image_ids = []

for path in flist:
    start = path.rfind("/")+1
    end = len(path)-4
    image_ids.append(path[start:end])
    
#image_ids

In [10]:
import pandas as pd

dataset2 = pd.read_csv("MovieGenre417.csv")
dataset2

,Unnamed: 0,index,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,0,0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,2,2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
2,11,11,112896,http://www.imdb.com/title/tt112896,Dracula: Dead and Loving It (1995),5.8,Comedy|Fantasy|Horror,https://images-na.ssl-images-amazon.com/images...
3,23,23,114168,http://www.imdb.com/title/tt114168,Powder (1995),6.5,Drama|Fantasy|Mystery,https://images-na.ssl-images-amazon.com/images...
4,26,26,114011,http://www.imdb.com/title/tt114011,Now and Then (1995),6.8,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...,...,...
5379,39322,40060,4508542,http://www.imdb.com/title/tt4508542,Nacida para ganar (2016),4.8,Comedy,https://images-na.ssl-images-amazon.com/images...
5380,39333,40071,1018706,http://www.imdb.com/title/tt1018706,CÌ£o Sem Dono (2007),6.9,Drama,https://images-na.ssl-images-amazon.com/images...
5381,39347,40085,4189294,http://www.imdb.com/title/tt4189294,Lego DC Comics: Batman Be-Leaguered (2014),6.8,Animation|Short|Action,https://images-na.ssl-images-amazon.com/images...
5382,39362,40100,98216,http://www.imdb.com/title/tt98216,Roller Blade Warriors: Taken by Force (1989),3.9,Action|Adventure|Sci-Fi,https://images-na.ssl-images-amazon.com/images...


# Data Preprocessing

In [11]:
y = []
indexlist = []
classes = tuple()
ids = dataset2.imdbId.values.tolist()
for image_id in image_ids:
    #print(dataset2["imdbId"])
    genres = tuple((dataset2[dataset2["imdbId"] == int(image_id)]["Genre"].values[0]).split("|"))
    if int(image_id) in ids:
        indexlist.append(image_id)
    y.append(genres)
    classes = classes + genres
mlb = MultiLabelBinarizer()
mlb.fit(y)
y = mlb.transform(y)
classes = set(classes)
classes = list(classes)
classes.sort()

In [12]:
y_df = pd.DataFrame(y, columns = classes, index = indexlist)
y_df = y_df[['Drama', 'Comedy', 'Romance', 'Action', 'Crime']]
classes = y_df.columns.tolist()
y_df

,Drama,Comedy,Romance,Action,Crime
2371315,1,0,0,0,1
2504404,1,0,0,0,1
363473,1,0,0,0,0
389778,1,0,0,0,0
42693,0,1,0,1,0
...,...,...,...,...,...
409904,0,0,0,1,0
106685,1,1,0,0,0
3375370,0,1,0,0,0
3331846,1,1,0,0,0


In [13]:
y_df_reset = y_df.reset_index()

shape = y_df_reset.shape[1]

index_value = []
genre_lst = []

for i in range(len(y_df_reset)):
    index_value.append(int(y_df_reset.loc[i,"index"]))
    temp_list = []
    for j in y_df_reset.columns[1:]:
        temp_list.append(y_df_reset.loc[i,j])
    genre_lst.append(temp_list)

df = pd.DataFrame(list(zip(index_value, genre_lst)),
               columns =['imdbId', 'genrelst'])

result = dataset2.merge(df, on="imdbId")
result

,Unnamed: 0,index,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster,genrelst
0,0,0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 1, 0, 0, 0]"
1,2,2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 1, 1, 0, 0]"
2,11,11,112896,http://www.imdb.com/title/tt112896,Dracula: Dead and Loving It (1995),5.8,Comedy|Fantasy|Horror,https://images-na.ssl-images-amazon.com/images...,"[0, 1, 0, 0, 0]"
3,23,23,114168,http://www.imdb.com/title/tt114168,Powder (1995),6.5,Drama|Fantasy|Mystery,https://images-na.ssl-images-amazon.com/images...,"[1, 0, 0, 0, 0]"
4,26,26,114011,http://www.imdb.com/title/tt114011,Now and Then (1995),6.8,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[1, 1, 1, 0, 0]"
...,...,...,...,...,...,...,...,...,...
5379,39322,40060,4508542,http://www.imdb.com/title/tt4508542,Nacida para ganar (2016),4.8,Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 1, 0, 0, 0]"
5380,39333,40071,1018706,http://www.imdb.com/title/tt1018706,CÌ£o Sem Dono (2007),6.9,Drama,https://images-na.ssl-images-amazon.com/images...,"[1, 0, 0, 0, 0]"
5381,39347,40085,4189294,http://www.imdb.com/title/tt4189294,Lego DC Comics: Batman Be-Leaguered (2014),6.8,Animation|Short|Action,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 1, 0]"
5382,39362,40100,98216,http://www.imdb.com/title/tt98216,Roller Blade Warriors: Taken by Force (1989),3.9,Action|Adventure|Sci-Fi,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 1, 0]"


In [14]:
for x in range(len(result)):
    tempY.append((int(result['imdbId'].iloc[x]),result['genrelst'].iloc[x]))

#tempY

# Train/Test Split

In [15]:
#setting the length of training data
length=int(len(flist)*training_size)
length

4307

In [16]:
#extracting the data about the images that are available
i=0
for filename in flist:
    name=int(filename.split('/')[-1][:-4])
    for z in tempY:
        if(z[0]==name):
            
            img = array(cv2.imread(filename))
            img = swapaxes(img, 2,0)
            img = swapaxes(img, 2,1)

            if(i<length):
                x_train.append(img)
                y_train.append(z[1])
                i+=1
            else:
                x_test.append(img)
                y_test.append(z[1])
                i+=1

In [17]:
#converting the data from lists to numpy arrays
x_train=asarray(x_train,dtype=float)
x_test=asarray(x_test,dtype=float)
y_train=asarray(y_train,dtype=float)
y_test=asarray(y_test,dtype=float)

In [18]:
#scaling down the RGB data
x_train /= 255
x_test /= 255

In [19]:
#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (4307, 3, 268, 182)
4307 train samples
1077 test samples


In [20]:
train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

In [21]:
# Metric calculation

def metric(scores, targets):
    """
    :param scores: the output the model predict
    :param targets: the gt label
    :return: OP, OR, OF1, CP, CR, CF1
    calculate the Precision of every class by: TP/TP+FP i.e. TP/total predict
    calculate the Recall by: TP/total GT
    """
    num, num_class = scores.shape
    gt_num = np.zeros(num_class)
    tp_num = np.zeros(num_class)
    predict_num = np.zeros(num_class)


    for index in range(num_class):
        score = scores[:, index]
        target = targets[:, index]

        gt_num[index] = np.sum(target == 1)
        predict_num[index] = np.sum(score >= 0.5)
        tp_num[index] = np.sum(target * (score >= 0.5))

    predict_num[predict_num == 0] = 1  # avoid dividing 0
    OP = np.sum(tp_num) / np.sum(predict_num) #OP (Overall Precision) is the ratio of the number of correctly predicted positive samples to the total number of positive predictions made by the model
    OR = np.sum(tp_num) / np.sum(gt_num) #OR (Overall Recall) is the ratio of the number of correctly predicted positive samples to the total number of positive samples in the ground truth.
    OF1 = (2 * OP * OR) / (OP + OR) #OF1 (Overall F1 Score) is the harmonic mean of precision and recall.

    return OP, OR, OF1

# Model 1: Resnet50

In [22]:
# Resnet50 model
from torchvision import models

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.resnet(x)
        return x

In [23]:
model = ResNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
from statistics import mean
import numpy as np

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)

        preds = torch.round(output)
            
        #acc_list = []
        #preds = torch.round(output)
        #for i in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for j in range(len(classes)):
        #        if target[i][j] == 1 or preds[i][j] == 1:
        #            denom += 1
        #            if preds[i][j] == target[i][j]:
        #                result+=1
        #    acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model.eval()
    test_loss = 0
    i = 0
    
    OP_final = 0
    OR_final = 0
    OF1_final = 0
    
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model(data)
        
        preds = torch.round(output)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        OP_final += OP
        OR_final += OR
        OF1_final += OF1

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP_final/i, OR_final/i, OF1_final/i))

for epoch in range(0, epochs):
    train(epoch)
    test()

<ipython-input-21-447663f93c68>:28: RuntimeWarning: invalid value encountered in double_scalars
  OF1 = (2 * OP * OR) / (OP + OR) #OF1 (Overall F1 Score) is the harmonic mean of precision and recall.


Train Epoch: 0 [0/4307 (0%)]	Loss: 0.696184 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [64/4307 (1%)]	Loss: 0.662022 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [128/4307 (3%)]	Loss: 0.650683 	OP: 0.428571	OR: 0.030000	OF1: 0.056075
Train Epoch: 0 [192/4307 (4%)]	Loss: 0.623011 	OP: 0.444444	OR: 0.039604	OF1: 0.072727
Train Epoch: 0 [256/4307 (6%)]	Loss: 0.588942 	OP: 0.583333	OR: 0.072917	OF1: 0.129630
Train Epoch: 0 [320/4307 (7%)]	Loss: 0.583333 	OP: 0.714286	OR: 0.148515	OF1: 0.245902
Train Epoch: 0 [384/4307 (9%)]	Loss: 0.575533 	OP: 0.600000	OR: 0.178218	OF1: 0.274809
Train Epoch: 0 [448/4307 (10%)]	Loss: 0.535635 	OP: 0.575758	OR: 0.206522	OF1: 0.304000
Train Epoch: 0 [512/4307 (12%)]	Loss: 0.548104 	OP: 0.702703	OR: 0.270833	OF1: 0.390977
Train Epoch: 0 [576/4307 (13%)]	Loss: 0.517862 	OP: 0.650000	OR: 0.285714	OF1: 0.396947
Train Epoch: 0 [640/4307 (15%)]	Loss: 0.551816 	OP: 0.604651	OR: 0.270833	OF1: 0.374101
Train Epoch: 0 [704/4307 (16%)]	Loss: 0.509067 	OP: 

<ipython-input-24-dc3ff23db847>:50: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()


# Model 2: DenseNet201

In [ ]:
import torchvision
import torch.nn as nn

model2 = torchvision.models.densenet201(pretrained=True)
num_ftrs = model2.classifier.in_features
model2.classifier = nn.Linear(num_ftrs, len(classes))

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model2.parameters(), lr=learning_rate)

In [ ]:
from statistics import mean
import numpy as np

def train(epoch):
    model2.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model2(data)

        preds = torch.round(output)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model2.eval()
    test_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model2(data)
        
        preds = torch.round(output)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

for epoch in range(0, epochs):
    train(epoch)
    test()

# Model 3: ResNet18

In [ ]:
from torchvision import models

class ResNet152(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet152, self).__init__()
        self.resnet = models.resnet152(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.resnet(x)
        return x

In [ ]:
model3 = ResNet152()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model3.parameters(), lr=learning_rate)

In [ ]:
from statistics import mean
import numpy as np

def train(epoch):
    model3.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model3(data)

        preds = torch.round(output)
            
        #acc_list = []
        #preds = torch.round(output)
        #for i in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for j in range(len(classes)):
        #        if target[i][j] == 1 or preds[i][j] == 1:
        #            denom += 1
        #            if preds[i][j] == target[i][j]:
        #                result+=1
        #    acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model3.eval()
    test_loss = 0
    i = 0
    
    OP_final = 0
    OR_final = 0
    OF1_final = 0
    
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model3(data)
        
        preds = torch.round(output)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        OP_final += OP
        OR_final += OR
        OF1_final += OF1

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP_final/i, OR_final/i, OF1_final/i))

for epoch in range(0, epochs):
    train(epoch)
    test()

# Custom CNN

In [ ]:
import torch.nn as nn

class CustomCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(CustomCNN, self).__init__()

        self.fc = nn.Linear(4, len(classes))

    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
model4 = CustomCNN()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model4.parameters(), lr=learning_rate)

In [ ]:
from statistics import mean
import numpy as np

def train(epoch):
    model4.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model4(data)

        preds = torch.round(output)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model4.eval()
    test_loss = 0
    i = 0
    
    OP_final = 0
    OR_final = 0
    OF1_final = 0
    
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model4(data)
        
        preds = torch.round(output)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        OP_final += OP
        OR_final += OR
        OF1_final += OF1

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP_final/i, OR_final/i, OF1_final/i))

for epoch in range(0, epochs):
    train(epoch)
    test()